# Recommendation Option 2

### Count Vector with only descriptive words

In [2]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

In [3]:
beers = pd.read_csv('data/corrected_cvec.csv')
beers.drop('Unnamed: 0',axis =1, inplace = True)
beers.head()

,ab,aback,abacus,abandon,abate,abates,abbey,abbeys,abbot,abbreviated,...,zestful,zesty,zilch,zing,zingy,zip,zippy,zombie,zone,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
beer_info = pd.read_csv('data/Beers_U.csv')[['Beer_Name','Brewery_Name','ABV','Type']]
beer_info.head()

,Beer_Name,Brewery_Name,ABV,Type
0,That's What She Said,Tree House Brewing Company,5.60,Milk / Sweet Stout
1,Triple Bag,Long Trail Brewing Co.,11.00,American Strong Ale
2,Great Lakes Devil's Pale Ale,Great Lakes Brewery,6.60,English Pale Ale
3,Hornswoggled,Cigar City Brewing,5.00,Irish Red Ale
4,Schlafly Raspberry Hefeweizen,The Schlafly Tap Room,4.10,Fruit / Vegetable Beer


In [5]:
beer_info['Beer_Name_Full'] = beer_info['Brewery_Name']+' : '+beer_info['Beer_Name']



In [6]:
print(beers.shape)
print(beer_info.shape)

(9490, 11751)
(9490, 5)


In [7]:
# Tegrity check.  just want to do an eyeball comparison of the first observation in each dataframe
beers.iloc[0].sort_values(ascending = False)[:15]

chocolate    37
milk         33
lactose      27
beer         24
stout        23
creamy       21
dark         18
light        18
roast        17
taste        17
head         16
coffee       16
malt         16
good         16
brown        15
Name: 0, dtype: int64

In [8]:
#These values are what I would expect to see in a Milk Stout.

In [9]:
# Manually searched through beer types to remove them I don't want a stout being recommended
# because a stout was the input and the word "stout" was used alot by reviewers when referring to
# "this stout" or "that stout"
beers.columns.contains('weizenbock')

False

In [10]:
['stout','porter','lager','pa',]

['stout', 'porter', 'lager', 'pa']

In [11]:
import pickle
filename = 'data/descriptive_words'
infile = open(filename,'rb')
desc_words = list(pickle.load(infile))
infile.close()

In [12]:
desc_words.remove(None)

In [13]:
desc_df = pd.SparseDataFrame(beers[desc_words], columns = desc_words)



In [14]:
#Cosign Similarity Matrix
cms = cosine_similarity(desc_df, desc_df)
cms.shape

(9490, 9490)

In [15]:
search_df = pd.SparseDataFrame(cms, columns = beer_info.Beer_Name_Full,index = beer_info.Beer_Name_Full)
search_df.head()

Beer_Name_Full,Tree House Brewing Company : That's What She Said,Long Trail Brewing Co. : Triple Bag,Great Lakes Brewery : Great Lakes Devil's Pale Ale,Cigar City Brewing : Hornswoggled,The Schlafly Tap Room : Schlafly Raspberry Hefeweizen,Brasserie Ellezelloise : Hercule Stout,Lagunitas Brewing Company : Censored,Brasserie de l'Abbaye des Rocs s.a. : Brasserie Des Rocs Triple Impériale,Cambridge Brewing Company : Arquebus,Temperance Beer Co. : Gatecrasher,...,New Belgium Brewing : Fat Tire Belgian White,3 Floyds Brewing Co. : Necron 99,Bierbrouwerij De Koningshoeven B.V. : Tilburg's Dutch Brown Ale,New Glarus Brewing Company : Laughing Fox,Trillium Brewing Company : Dialed In (w/ Chardonnay & Gewürztraminer Juice),BrewDog : Riptide,Atlantic Brewing Company / Mainely Meat BBQ : New Guy IPA,Les Brasseurs de Gayant : Saint Landelin La Divine,Hill Farmstead Brewery : George,Köstritzer Schwarzbierbrauerei GmbH & Co. : Köstritzer Schwarzbier
Beer_Name_Full,,,,,,,,,,,,,,,,,,,,,
Tree House Brewing Company : That's What She Said,1.000000,0.736531,0.718741,0.661090,0.376503,0.820349,0.644814,0.715760,0.564252,0.623820,...,0.468975,0.596483,0.727281,0.639998,0.503012,0.807515,0.633380,0.611481,0.628388,0.764409
Long Trail Brewing Co. : Triple Bag,0.736531,1.000000,0.812535,0.716732,0.439372,0.773826,0.748451,0.854697,0.680017,0.659336,...,0.544498,0.667275,0.835776,0.735636,0.550431,0.771936,0.687228,0.706137,0.717532,0.778120
Great Lakes Brewery : Great Lakes Devil's Pale Ale,0.718741,0.812535,1.000000,0.683575,0.461467,0.724301,0.779551,0.774411,0.649968,0.743934,...,0.587473,0.650180,0.779393,0.756032,0.582788,0.715938,0.736730,0.731529,0.661513,0.753829
Cigar City Brewing : Hornswoggled,0.661090,0.716732,0.683575,1.000000,0.398351,0.644281,0.746014,0.665790,0.610890,0.591269,...,0.551239,0.627674,0.677598,0.744004,0.542614,0.597216,0.619352,0.668453,0.540601,0.656180
The Schlafly Tap Room : Schlafly Raspberry Hefeweizen,0.376503,0.439372,0.461467,0.398351,1.000000,0.387944,0.458488,0.411814,0.449250,0.444262,...,0.453010,0.417432,0.380029,0.485009,0.423323,0.367749,0.508650,0.479916,0.283192,0.368980


In [16]:
search_df.sort_index(axis =0, inplace = True)
search_df.sort_index(axis =1, inplace = True)

In [17]:
search_df.head()

Beer_Name_Full,(512) Brewing Company : (512) Black IPA,(512) Brewing Company : (512) Bruin,(512) Brewing Company : (512) Cascabel Cream Stout,(512) Brewing Company : (512) IPA,(512) Brewing Company : (512) Pecan Porter,(512) Brewing Company : (512) Whiskey Barrel Aged Double Pecan Porter,(512) Brewing Company : (512) Wit,10 Barrel Brewing Co. : Apocalypse IPA,10 Barrel Brewing Co. : Cucumber Crush,10 Barrel Brewing Co. : DUB Double India Pale Ale,...,pFriem Family Brewers : Pilsner,von Trapp Brewing : Von Trapp Bohemian Pilsner,von Trapp Brewing : Von Trapp Dunkel Lager,von Trapp Brewing : Von Trapp Golden Helles Lager,von Trapp Brewing : Von Trapp India Pale Lager,von Trapp Brewing : Von Trapp Trosten Lager,von Trapp Brewing : Von Trapp Vienna Lager,Žatecký Pivovar : Žatec,Žatecký Pivovar : Žatec Dark Lager,Ölvisholt Brugghús : Lava
Beer_Name_Full,,,,,,,,,,,,,,,,,,,,,
(512) Brewing Company : (512) Black IPA,1.000000,0.720668,0.665163,0.609810,0.854042,0.834410,0.560342,0.645899,0.497222,0.587318,...,0.541867,0.541179,0.733931,0.572497,0.586441,0.798598,0.628392,0.592887,0.785781,0.861460
(512) Brewing Company : (512) Bruin,0.720668,1.000000,0.597743,0.571205,0.788722,0.773205,0.488760,0.555255,0.432165,0.514877,...,0.428708,0.425103,0.780241,0.487279,0.449153,0.802588,0.581908,0.486694,0.828852,0.742055
(512) Brewing Company : (512) Cascabel Cream Stout,0.665163,0.597743,1.000000,0.552105,0.729769,0.742070,0.495874,0.578749,0.467393,0.543185,...,0.460400,0.458593,0.615488,0.493551,0.464037,0.647919,0.556827,0.523223,0.621493,0.721112
(512) Brewing Company : (512) IPA,0.609810,0.571205,0.552105,1.000000,0.685631,0.631188,0.755101,0.866217,0.550303,0.819087,...,0.665758,0.714526,0.653867,0.715507,0.769165,0.663559,0.785732,0.745776,0.604592,0.614402
(512) Brewing Company : (512) Pecan Porter,0.854042,0.788722,0.729769,0.685631,1.000000,0.860090,0.585571,0.661647,0.490378,0.631568,...,0.531935,0.536882,0.790804,0.579206,0.550591,0.842357,0.674338,0.593956,0.837968,0.862722


In [18]:
# Set the full name to the index so it can be use to search.
beer_info.set_index('Beer_Name_Full', inplace = True)

In [19]:
# Pickle dataframe from later
search_df.to_pickle('./Data/modeldata/dataframe_DW')

In [20]:
def recommend(beer_name, df = search_df, info = beer_info):
    top_ten = df.loc[beer_name].sort_values(ascending = False)[1:11].index
    return info.loc[top_ten].set_index('Beer_Name')

In [21]:
def search_beer(name):
    return beer_info[beer_info.index.str.contains(name,case = False, regex = True)]


### Start from the Bottom
Lets from the bottom of the beer bucket.  Unfortunately, I have a friend who loves Busch Beer (and hates getting iced). Lets see what <strike>crap</strike> _"beers"_ are recommended for him.  

In [22]:
# Find Busch
search_beer('Anheuser-Busch : Busch')

,Beer_Name,Brewery_Name,ABV,Type
Beer_Name_Full,,,,
Anheuser-Busch : Busch Light,Busch Light,Anheuser-Busch,4.10,Light Lager
Anheuser-Busch : Busch Ice,Busch Ice,Anheuser-Busch,5.90,American Adjunct Lager
Anheuser-Busch : Busch Beer,Busch Beer,Anheuser-Busch,4.30,American Adjunct Lager


In [23]:
recommend('Anheuser-Busch : Busch Beer')

,Brewery_Name,ABV,Type
Beer_Name,,,
Miller 64,Miller Brewing Co.,2.80,Light Lager
Natural Light,Anheuser-Busch,4.20,Light Lager
Palma Louca Pale Pilsner,Cervejaria Kaiser,4.50,German Pilsener
Michelob Light,Anheuser-Busch,4.30,Light Lager
Famosa (Gallo),"Cervecería Centro Americana, S.A.",5.00,American Adjunct Lager
Busch Light,Anheuser-Busch,4.10,Light Lager
Miller Lite,Miller Brewing Co.,4.17,Light Lager
Kirin Light Beer,"Kirin Brewery Company, Limited",3.20,Light Lager
Southpaw Light,Miller Brewing Co.,5.00,Light Lager


In [24]:
# My brother became a hipster in college
recommend('Pabst Brewing Company : Pabst Blue Ribbon (PBR)')

,Brewery_Name,ABV,Type
Beer_Name,,,
Tsingtao,"Tsingtao Brewery Co., Ltd.",4.80,American Adjunct Lager
Baltika #5 Gold,Baltika Breweries,5.30,Euro Pale Lager
Cristal,Union De Cervecerias Peruanas Backus y Johnston,5.00,American Adjunct Lager
Carib Lager,Carib Brewery Limited,5.20,American Adjunct Lager
Kalik,Commonwealth Brewery Ltd.,5.00,American Adjunct Lager
Quilmes,Cervecería y Maltería Quilmes / Grupo Bemberg,4.90,American Adjunct Lager
Natural Ice,Anheuser-Busch,5.90,American Adjunct Lager
Molson Golden,Molson Coors Canada,5.00,American Pale Lager
Famosa (Gallo),"Cervecería Centro Americana, S.A.",5.00,American Adjunct Lager


In [25]:
# this ones for you Camouflage
recommend('Anheuser-Busch : Bud Light')

,Brewery_Name,ABV,Type
Beer_Name,,,
Miller 64,Miller Brewing Co.,2.80,Light Lager
Natural Light,Anheuser-Busch,4.20,Light Lager
Busch Light,Anheuser-Busch,4.10,Light Lager
Miller Lite,Miller Brewing Co.,4.17,Light Lager
Heineken Premium Light Lager,Heineken Nederland B.V.,4.20,Light Lager
Coors Light,Coors Brewing Company (Molson-Coors),4.20,Light Lager
Michelob Ultra,Anheuser-Busch,4.20,Light Lager
Labatt Blue Light,Labatt Brewing Company Ltd.,4.00,Light Lager
Keystone Light,Coors Brewing Company (Molson-Coors),4.10,Light Lager


### Something a little more traditional.

My dad likes Stella Artois.  (I like how The Dude pronounces it 'Stella Arttoes')
Lets see if we can find some similar European Pale Lagers...

In [26]:
# Find Stella
search_beer('Stella Artois')

,Beer_Name,Brewery_Name,ABV,Type
Beer_Name_Full,,,,
Stella Artois : Club De Stella Artois,Club De Stella Artois,Stella Artois,5.90,Euro Pale Lager
Stella Artois : Stella Artois,Stella Artois,Stella Artois,5.00,Euro Pale Lager


In [27]:
recommend('Stella Artois : Stella Artois')

,Brewery_Name,ABV,Type
Beer_Name,,,
Efes Pilsener,Anadolu Efes Brewery,5.00,German Pilsener
Kronenbourg 1664,Brasseries Kronenbourg,5.50,Euro Pale Lager
DAB Original,Dortmunder Actien Brauerei / DAB,5.00,Dortmunder / Export Lager
Birra Moretti,Birra Moretti (Heineken),4.60,Euro Pale Lager
Straub American Lager,Straub Brewery,4.30,American Adjunct Lager
Bavik Pils / Lager,De Brabandere,5.20,German Pilsener
Baltika #7 Export,Baltika Breweries,5.40,Dortmunder / Export Lager
Saranac Bohemian Pilsener,Matt Brewing Company / Saranac Brewery,4.80,Czech Pilsener
Warsteiner Premium German Pilsener,Warsteiner Brauerei,4.80,German Pilsener


In [28]:
# Grandma likes Heineken
recommend('Heineken Nederland B.V. : Heineken Pilsener')
# These results are quick interesting.

,Brewery_Name,ABV,Type
Beer_Name,,,
Lawnmower Lager,Caldera Brewing Company,3.90,American Adjunct Lager
Warsteiner Premium German Pilsener,Warsteiner Brauerei,4.80,German Pilsener
Saranac Kölsch,Matt Brewing Company / Saranac Brewery,5.00,Kölsch
Schlitz Gusto (Classic 1960's Formula),JOS. Schlitz Brewing Co. (Pabst),4.70,American Adjunct Lager
Leinenkugel's Original,Jacob Leinenkugel Brewing Company,4.60,American Adjunct Lager
Steinlager,New Zealand Breweries Limited,4.80,Euro Pale Lager
Yuengling Premium Beer,Yuengling Brewery,4.40,American Pale Lager
DAB Original,Dortmunder Actien Brauerei / DAB,5.00,Dortmunder / Export Lager
Henry Weinhard's Private Reserve,Blitz-Weinhard Brewing Co.,4.80,American Pale Lager


### Lets Switch it up!
And look at why this recommended was created.

I have some friends, they really like Jai Alai IPA from Cigar City. I agree with them, its a pretty good.

In [29]:
recommend('Cigar City Brewing : Jai Alai IPA')
# some solid recommendations

,Brewery_Name,ABV,Type
Beer_Name,,,
Hop Henge Experimental IPA,Deschutes Brewery,9.30,American Double / Imperial IPA
Deviant Dale's IPA,Oskar Blues Grill & Brew,8.00,American IPA
White Knuckle Double IPA,Marin Brewing Company,8.00,American Double / Imperial IPA
Apex,Bear Republic Brewing Co.,8.25,American Double / Imperial IPA
Stateside IPA,Mikkeller ApS,7.00,American IPA
Big Eye IPA,Ballast Point Brewing Company,7.00,American IPA
Hop Head Double India Pale Ale,Tree Brewing,8.30,American Double / Imperial IPA
Wildcat India Pale Ale,Snoqualmie Falls Brewing Company & Taproom,6.60,American IPA
Café Racer 15,Bear Republic Brewing Co.,9.75,American Double / Imperial IPA


In [30]:
recommend('New Belgium Brewing : La Folie')

,Brewery_Name,ABV,Type
Beer_Name,,,
Cuvée Des Jacobins Rouge,Brouwerij Bockor N.V. / Brouwerij Omer Vander ...,5.50,Flanders Red Ale
Panil Barriquée (Italy Version),Panil,8.00,Flanders Red Ale
Panil Barriquée (Sour Version),Panil,8.00,Flanders Red Ale
Barrel Select Batch 1,Captain Lawrence Brewing Co.,7.50,American Wild Ale
Silly Sour,Brasserie de Silly S.A.,5.50,Saison / Farmhouse Ale
The Wanderer,The Bruery,8.00,American Wild Ale
Cascade Sang Rouge,Cascade Brewing / Raccoon Lodge & Brewpub,8.06,American Wild Ale
Oud Bruin,Brouwerij 't Verzet / Café Loca,6.00,Flanders Oud Bruin
La Roja,Jolly Pumpkin Artisan Ales,7.20,American Wild Ale


In [31]:
recommend("New Belgium Brewing : Hop Kitchen - APA (FOCOllaboration W/ Odell Brewing)")


,Brewery_Name,ABV,Type
Beer_Name,,,
Anti-Hero IPA,Revolution Brewing,6.70,American IPA
Great Lakes Commodore Perry IPA,Great Lakes Brewing Co.,7.50,English India Pale Ale (IPA)
Nectar IPA,Humboldt Brewing Co.,6.70,American IPA
Fat Head's Head Hunter IPA,Fat Head's Brewery & Saloon,7.50,American IPA
Fathom India Pale Lager,Ballast Point Brewing Company,7.00,American Double / Imperial Pilsner
Columbus IPA,Columbus Brewing Company,6.30,American IPA
Red Chair NWPA,Deschutes Brewery,6.20,American Pale Ale (APA)
Marble IPA,Marble Brewery,6.80,American IPA
Citra Extra Pale Ale,Knee Deep Brewing Company,7.00,American Pale Ale (APA)


In [32]:
# Sculpin is good but often overpriced (in my opinion)
recommend('Ballast Point Brewing Company : Sculpin IPA')

,Brewery_Name,ABV,Type
Beer_Name,,,
Workhorse IPA,Laurelwood Public House & Brewery,7.50,American IPA
Dreadnaught IPA,3 Floyds Brewing Co.,9.50,American Double / Imperial IPA
Pliny The Elder,Russian River Brewing Company,8.00,American Double / Imperial IPA
Apex,Bear Republic Brewing Co.,8.25,American Double / Imperial IPA
Lagunitas Sucks (Brown Shugga Substitute Ale),Lagunitas Brewing Company,7.85,American Double / Imperial IPA
Deviant Dale's IPA,Oskar Blues Grill & Brew,8.00,American IPA
Hop Henge Experimental IPA,Deschutes Brewery,9.30,American Double / Imperial IPA
Abner,Hill Farmstead Brewery,8.20,American Double / Imperial IPA
Hopslam Ale,"Bell's Brewery, Inc.",10.00,American Double / Imperial IPA


In [33]:
# A beer that you litterally drink 2-3oz at a time
recommend('Dogfish Head Craft Brewery : 120 Minute IPA')

,Brewery_Name,ABV,Type
Beer_Name,,,
Belzebuth,Brasserie Grain d' Orge (Brasserie Jeanne d'Ar...,13.00,Belgian Strong Pale Ale
Shipyard Smashed Pumpkin (Pugsley's Signature Series),Shipyard Brewing Company,9.00,Pumpkin Ale
Dominion Millennium Ale,Old Dominion Brewing Company,10.50,English Barleywine
Hop Juice Double IPA,Oggi's Pizza & Brewing Co.,9.40,American Double / Imperial IPA
Bavarian-Styled Doppel Weizen,Lagunitas Brewing Company,8.70,Weizenbock
Founders Devil Dancer,Founders Brewing Company,12.00,American Double / Imperial IPA
Scaldis Amber,Brasserie Dubuisson Frères sprl,12.00,Belgian Strong Pale Ale
Drake's Denogginizer,Drake's Brewing Co.,9.75,American Double / Imperial IPA
Druid Fluid,"Middle Ages Brewing Co., Ltd.",9.50,American Barleywine


In [34]:
# Beer that made me really like Pale Ales
recommend("BJ's Restaurant & Brewhouse : Piranha Pale Ale")

,Brewery_Name,ABV,Type
Beer_Name,,,
Chesapeake Pale Ale,AleWerks Brewing Company,5.30,American Pale Ale (APA)
IPA,Rubicon Brewing Company,6.50,American IPA
Piranha Pale Ale,BJ's Chicago Pizza & Brewery Inc.,5.70,American Pale Ale (APA)
Pipers Pale Ale,Dunedin Brewery,6.00,American Pale Ale (APA)
Red Racer Classic Pale Ale,Central City Brewers + Distillers,5.00,American Pale Ale (APA)
Poleeko Pale Ale,Anderson Valley Brewing Company,5.00,American Pale Ale (APA)
Dundee Pale Ale,Genesee Brewing Co. / Dundee Brewing Co.,5.30,American Pale Ale (APA)
Back Yard India Pale Ale,Cooperstown Brewing Company,6.10,English India Pale Ale (IPA)
Brooklyn East India Pale Ale,Brooklyn Brewery,6.90,English India Pale Ale (IPA)


In [35]:
# Roommate says this brown ale
recommend('Upslope Brewing Company - Lee Hill : Brown Ale')

,Brewery_Name,ABV,Type
Beer_Name,,,
George,Hill Farmstead Brewery,6.00,American Brown Ale
Peter Brown Tribute Ale,Bear Republic Brewing Co.,6.30,American Brown Ale
Face Down Brown,Telluride Brewing Co.,5.70,American Brown Ale
Baird Angry Boy Brown Ale,Baird Brewing Company / Fishmarket Taproom,6.20,English Brown Ale
Nimbus Nut Brown Ale,Nimbus Brewing Company,5.00,English Brown Ale
Davy Brown Ale,Figueroa Mountain Brewing Co.,6.00,American Brown Ale
Jackie Brown,Mikkeller ApS,5.90,American Brown Ale
The Brown Note,Against The Grain Brewery & Smokehouse,5.00,English Brown Ale
Gnarly Brown,MadTree Brewing,7.00,American Brown Ale
